In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt


from models.unet_resnet18 import build_model
from src.dataset import SteelDefectDataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Cargar modelo
model = build_model().to(DEVICE)
model.load_state_dict(torch.load("outputs/checkpoints/best_model.pth", map_location=DEVICE))
model.eval()

# Dataset para obtener imágenes reales
import pandas as pd
df = pd.read_csv("data/val_split.csv")
df.columns = ["ImageId", "ClassId", "EncodedPixels"]

dataset = SteelDefectDataset(df, "data/train_images")

# Tomar una imagen al azar
img_tensor, mask_tensor = dataset[10]

img = img_tensor.squeeze().numpy()
mask_real = mask_tensor.numpy()

# Predicción
inp = img_tensor.unsqueeze(0).to(DEVICE)
with torch.no_grad():
    pred = model(inp).cpu().numpy()[0]

pred = (pred > 0.5).astype(np.uint8)

# Mostrar
fig, axs = plt.subplots(3, 5, figsize=(18,10))

axs[0,0].imshow(img, cmap="gray"); axs[0,0].set_title("Entrada")

for i in range(4):
    axs[1,i].imshow(mask_real[i], cmap="hot")
    axs[1,i].set_title(f"Máscara real {i+1}")

for i in range(4):
    axs[2,i].imshow(pred[i], cmap="hot")
    axs[2,i].set_title(f"Predicción {i+1}")

plt.show()


ModuleNotFoundError: No module named 'models'